# Import Libraries

In [ ]:
from dash import Dash, html, dcc, callback, Output, Input, State
import dash_bootstrap_components as dbc
import plotly.express as px
import plotly.graph_objects as go

import pandas as pd
import numpy as np

import json
import os

# Initialize App 

In [ ]:
app = Dash(
    __name__,
    external_stylesheets=[
        dbc.themes.BOOTSTRAP,
        dbc.icons.FONT_AWESOME
    ]
)

# Load Data

In [ ]:
athlete_events_df = pd.read_csv("./assets/athlete_events.csv")
noc_regions = pd.read_csv("./assets/noc_regions.csv")

# athlete_events_df = athlete_events_df[athlete_events_df["Medal"].notna()]

# athlete_events_df.groupby(["Weight"])["Weight"].count().reset_index(name="height_count").sort_values(by=["height_count"], ascending=False)

# athlete_events_df.groupby(["Name", "Medal"])["Medal"].count()
# noc_medals_df = athlete_events_df.merge(noc_regions, on="NOC", how="left").rename(columns={"region":"Region"})
# noc_medals_df = noc_medals_df.groupby(["NOC", "Medal"])["Medal"].count().unstack(fill_value=0).stack().reset_index(name="Medal_Count")
# noc_medals_df["United States of America"]
# noc_medals_df[noc_medals_df["NOC"] == "USA"]

# athlete_events_df[(athlete_events_df["NOC"] == "USA") & (athlete_events_df["Medal"].notna())]
# athlete_events_df.sort_values(by=["Weight"], ascending=False)
# athlete_events_df.drop_duplicates(subset=["Sport"])

# Total Medal Count by Height and Weight, grouped by Sport (Heatmap Figure)

In [ ]:
# Group medal counts by height and weight
height_weight_medal_df = athlete_events_df.dropna(subset=["Height", "Weight"])
height_weight_medal_df = height_weight_medal_df.groupby(["Sport", "Height", "Weight"])["Medal"].count().reset_index(name="Medal_Count")

# List of medal counts by height and weight for each sport
sports_df_list = {col:pd.pivot_table(sport, index="Height", columns="Weight", values="Medal_Count").fillna(0) for (col, sport) in height_weight_medal_df.groupby('Sport')}

# Initial heatmap figure
heatmap_fig = go.Figure(
    data=go.Heatmap(
        x=sports_df_list["Archery"].columns,
        y=sports_df_list["Archery"].index,
        z=sports_df_list["Archery"].values,
        visible=True
    )
)
heatmap_fig.update_layout(
    title="Total Medals Won by Height and Weight in Archery"
)

# Add heatmap traces for each sport
for key in sports_df_list.keys():
    heatmap_fig.add_traces(
        data=go.Heatmap(
            x=sports_df_list[key].columns,
            y=sports_df_list[key].index,
            z=sports_df_list[key].values,
            visible=False
        )
    )

# Create dropdown option for each sport
sports_options = []
for i, key in enumerate(sports_df_list.keys()):
    visible_list = [False] * len(sports_df_list.keys())
    visible_list[i] = True
    
    sports_options.append(dict(
        label=key,
        method="update",
        args=[
            {"visible":visible_list},
            {"title":f"Total Medals Won by Height and Weight in {key}"}
        ]
    ))
heatmap_fig.update_layout(
    autosize=False,
    xaxis=dict(title="Weight (kg)"),
    yaxis=dict(title="Height (cm)"),
    updatemenus=[dict(
        active=0,
        buttons=sports_options,
    )]
)

heatmap_fig.show()

# Interactive Map (Choropleth Figure)

In [ ]:


# Example Dataset and Choropleth Figure
df = pd.read_csv("./assets/aid_data.csv")
df = df.dropna()

# Dataframe for sum of donations given grouped by country
donors_df = df.rename(columns={"donor":"country"}).sort_values(by=["commitment_amount_usd_constant"], ascending=True).groupby(["country"])["commitment_amount_usd_constant"].sum().reset_index(name="total_donated")

# Dataframe for sum of donations received grouped by country
recipients_df = df.rename(columns={"recipient":"country"}).groupby(["country"])["commitment_amount_usd_constant"].sum().reset_index(name="total_received")

# Dataframe for net donations grouped by country
combined_df = pd.merge(donors_df, recipients_df, how="outer", on="country").fillna(0)
combined_df["net"] = combined_df["total_donated"] - combined_df["total_received"]

# Choropleth figure
fig_q1 = px.choropleth(combined_df,
                        title="Net Donations by Country",
                        locations="country",
                        locationmode="country names",
                        color="net",
                        color_continuous_scale="PiYg",
                        hover_name="country",
                        range_color=[-4e6, 4e6],
                        projection="natural earth")

fig_q1.show()

# Dash Layout

In [ ]:
fig_q1.update_layout(clickmode='event+select')

country = None

app.layout = html.Div([
    # dcc.Graph(
    #     id="test-map",
    #     figure=fig_q1
    # ),
    dcc.Graph(
        id="heatmap",
        figure=heatmap_fig
    ),
    dbc.Modal(
        [
            dbc.ModalHeader(dbc.ModalTitle(id="modal-text", children=[
                "Placeholder"
            ]))
        ],
        id="modal-sm",
        size="sm",
        is_open=False,
    )
])

# @callback(
#     Output('hover-data', 'children'),
#     Input('test-map', 'clickData')
# )
# def display_click_data(clickData):
#     # return json.dumps(clickData, indent=2)["points"][0]
#     if clickData:
#         country = clickData["points"][0]["location"]
#         return clickData["points"][0]["location"]
#     country = None
#     return None

@callback([
        Output("modal-text", "children"),
        Output("modal-sm", "is_open")
    ],
    [
        Input("test-map", "clickData")
    ],
    [
        State("modal-sm", "is_open")
    ]
)
def update_modal(clickData, is_open):
    # print(clickData)
    if clickData:
        return clickData["points"][0]["location"], not is_open
    return None, is_open
    # return clickData

# def toggle_modal(n1, is_open):
#     if n1:
#         print(is_open)
#         return not is_open
#     print(is_open)
#     return is_open

# app.callback(
#     Output("modal-sm", "is_open"),
#     Input("open-sm", "n_clicks"),
#     State("modal-sm", "is_open"),
# )(toggle_modal)

# Run Dash App

In [ ]:
app.run_server(debug=True)